# **Segmenting & Clustering Neighborhoods in Toronto**

The project includes **scraping the Wikipedia page** for the postal codes of Canada and then **process and clean the data** for the clustering. The clustering is carried out by K Means and the clusters are plotted using the Folium Library. The Boroughs containing the name 'Toronto' in it are first plotted and then clustered and plotted again.

### Installing Packages

In [1]:
!pip install geopy
!pip install beautifulsoup4
!pip install lxml

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import random #random number generation
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from IPython.display import display_html

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Folium Installed')
from bs4 import BeautifulSoup
print('BeautifulSoup Installed')

print('Libraries imported.')

Folium Installed
BeautifulSoup Installed
Libraries imported.


## **Part 1: Web Scrapping Data**

Building code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [3]:
#Displaying html table

src = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(src,'lxml')

tab = str(soup.table)
#display_html(tab, raw = True) 

#Converting html table to pandas dataframe
read_tab = pd.read_html(tab)
df = read_tab[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.shape

(180, 3)

## **Part 2: Data Pre-processing**



1. We are to only process the cells that have an assigned boroug and ignore cells with a borough that is *Not assigned*.
2. Combine rows with same postal code area
3. Assigning borough name to neighborhood name if the neighborhood name is not assigned 



In [5]:
df.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [0]:
# Removing rows which don't have borough names assigned
df1 = df[df['Borough'] != 'Not assigned']

In [0]:
#df1.Borough.value_counts()

In [0]:
#Combine rows with same postal code area
df2 = df1.groupby(['Postal Code','Borough'],
                               sort=False).agg(', '.join)
df2.reset_index(inplace=True)

In [9]:
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [0]:
#Assigning borough name to neighborhood name if the neighborhood name is not assigned 
df2['Neighborhood'] = np.where(df2['Neighborhood'] == 'Not assigned',
                               df2['Borough'], df2['Neighborhood'])

In [11]:
df2.shape

(103, 3)

## **Part 3: Clustering**

Importing csv file containing latitudes and longitudes for Postal Codes in Canada from https://cocl.us/Geospatial_data

In [12]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Joinig this data set with the previously cleaned postal codes list
df3 = pd.merge(df2, coordinates, on=['Postal Code'])
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df3['Borough'].unique()),
        df3.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### **Use geopy library to get the latitude and longitude values of Toronto.**

In [15]:
#Creating geographical coordinates for Toronto, ON

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### **Creating a map of Toronto with the Latitudes & Longitudes**

In [16]:
#Creating Map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=10)

#Adding Markers on to the map
for lat, lng, label in zip(df3['Latitude'],df3['Longitude'],df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### **k-Means to CLuster the Neighborhoods**

In [17]:
k = 5 #No. of clusters
toronto_clustering = df3.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df3.insert(0, 'Cluster Labels', kmeans.labels_)
#pr)int(df3.head()

#Creating Map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(df3['Latitude'],
                                           df3['Longitude'],
                                           df3['Neighborhood'],
                                           df3['Cluster Labels']):
  
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters